In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 트랜스포머
!pip install sentencepiece transformers accelerate einops -q
!pip install transformers -q

# 번역
!pip install googletrans==3.1.0a0 -q

In [ ]:
import re  # 텍스트 전처리
import time  # 생성 시간 처리
import torch  # 파이토치
from transformers import AutoModelForCausalLM, AutoTokenizer  # 모델, 토큰 가져오기
from googletrans import Translator  # 번역

my_path = '/content/drive/MyDrive/my_ws/project/Final/phi2'

trans = Translator()

torch.set_default_device("cuda")
model = AutoModelForCausalLM.from_pretrained(my_path,
                                             torch_dtype=torch.float32,
                                             device_map="cuda",
                                             trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(my_path,
                                          trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 번역 확인
result = trans.translate("저는 ZIGISPACE라는 블로그를 운영하는 ZIGI라고 합니다.",dest='en', src='ko')
print(f"원  문({result.src}) : {result.origin}")
print(f"번역문({result.dest}) : {result.text}")

원  문(ko) : 저는 ZIGISPACE라는 블로그를 운영하는 ZIGI라고 합니다.
번역문(en) : My name is ZIGI and I run a blog called ZIGISPACE.


In [ ]:
def generate(prompt: str)-> str :

    s = time.time()  # 시간 시작

    input_split_text(prompt)  # '.' 단위로 나누기
    result = trans.translate(prompt ,dest='en', src='ko')  # 한글을 영어로
    prompt = result.text

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)  # 인풋 토큰화
    num_input_tokens = inputs['input_ids'].shape[1]  # 인풋 토큰화 사이즈

    outputs = model.generate(**inputs, max_length=num_input_tokens+80)  # 인풋 토큰화 생성 사이즈 + 80 만큼 더 만들기

    # 아웃풋 토큰화를 영어로 - 인풋 토큰화 사이즈(결과에 인풋이 들어간 것 빼기)
    completion = tokenizer.batch_decode(outputs[:, num_input_tokens:])[0]

    elapsed = time.time() - s  # 시간 계산

    num_total_tokens = outputs.shape[1]  # 아웃풋 토큰화 사이즈
    num_output_tokens = float(num_total_tokens) - num_input_tokens  # 실제 아웃풋 토큰화 사이즈
    speed = num_output_tokens / elapsed  # 최종 시간 계산

    # 부가 요소 시간, 토큰 사이즈 출력
    print(f"Took {round(elapsed,1)} seconds to generate {num_input_tokens}/{num_output_tokens} new tokens at speed {round(speed, 1)} tokens/seconds")
    print(f"---------------------------------------------")

    # 아웃풋 한글로 바꾸기
    completion_ko = trans.translate(completion.split("<|endoftext|>")[0], dest='ko', src='en').text

    # 아웃풋 '.' 단위로 나누기
    output_split_text(completion_ko)


def output_split_text(text: str):

    # 개행 제거
    text = text.replace('\n', ' ')

    # 마침표를 포함하여 분할
    sentences = re.split('(?<=\.) ', text)

    # 각 문장의 앞뒤 공백 제거
    sentences = [sentence.strip() for sentence in sentences if sentence]

    # 마지막 문장이 완전하지 않으면 제거
    if not sentences[-1].endswith('.'):
        sentences = sentences[:-1]

    # 리스트 출력
    for i, sentence in enumerate(sentences):
        print(f"문장 {i+1}: {sentence}")


def input_split_text(text: str):

    # 마침표를 포함하여 분할
    sentences = re.split('(?<=\.) ', text)

    # 각 문장의 앞뒤 공백 제거
    sentences = [sentence.strip() for sentence in sentences if sentence]

    # 리스트 출력
    for i, sentence in enumerate(sentences):
        print(f"{sentence}")


In [ ]:
en_text = "군 복무를 마치고 복학한 '만섭'은 학교에 족구장이 사라졌다는 걸 알게 된다. 그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다."
en_completion = generate(en_text)
print()
en_text += " 그러나 대학 측은 법원이 필요하지 않다며 이를 거부하고 있다."
en_completion = generate(en_text)
print()
en_text += " 대학팀은 도움을 주기로 하지만 조건이 있다."
en_completion = generate(en_text)
print()
en_text += " 대학 대표팀은 김승우, 김승준, 김승현 3명으로 구성됐다."
en_completion = generate(en_text)

군 복무를 마치고 복학한 '만섭'은 학교에 족구장이 사라졌다는 걸 알게 된다.
그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다.
Took 5.5 seconds to generate 46/80.0 new tokens at speed 14.5 tokens/seconds
---------------------------------------------
문장 1: 그러나 대학 측은 법원이 필요하지 않다며 이를 거부하고 있다.
문장 2: 이는 학생들이 법원이 신체적, 정신적 안녕에 필수적이라고 믿기 때문에 학생과 대학 사이의 갈등으로 이어집니다.
문장 3: 영화는 또한 스포츠계의 성 불평등 문제를 조명합니다.
문장 4: 한국에서는 스포츠가 남자의 것이라는 오랜 믿음이 있습니다.

군 복무를 마치고 복학한 '만섭'은 학교에 족구장이 사라졌다는 걸 알게 된다.
그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다.
그러나 대학 측은 법원이 필요하지 않다며 이를 거부하고 있다.
Took 6.5 seconds to generate 63/80.0 new tokens at speed 12.3 tokens/seconds
---------------------------------------------
문장 1: 한편, 학교 배구팀은 새로운 코치를 찾기 위해 고군분투하고 있습니다.
문장 2: 그들은 대학 배구팀에 도움을 요청하기로 결정합니다.
문장 3: 대학팀은 도움을 주기로 하지만 조건이 있다.
문장 4: 그들은 자신의 연습을 위해 학교 배구장을 사용하고 싶어합니다.
문장 5: 학교 팀은 이에 동의하고 함께 연습을 시작합니다.

군 복무를 마치고 복학한 '만섭'은 학교에 족구장이 사라졌다는 걸 알게 된다.
그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다.
그러나 대학 측은 법원이 필요하지 않다며 이를 거부하고 있다.
대학팀은 도움을 주기로 하지만 조건이 있다.
Took 5.4 seconds to 